In [5]:
import openai
from langchain.llms import OpenAI
from langchain.chains import SequentialChain, SimpleSequentialChain, ConversationChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.memory import ConversationBufferWindowMemory, ConversationBufferMemory

# other imports
from secret import API_KEY
# import os

#### Basic llms longchain

In [11]:
llm = OpenAI(openai_api_key = API_KEY,temperature=0.3)
answer = llm.predict("What is the basic of Programming?")
print(answer.strip())

The basic of programming is the ability to write code that can be used to create programs, applications, and websites. It involves writing code in a specific language, such as C++, Java, or Python, and using it to create a program that can be used to perform a specific task.
